In [1]:
import json
import stanza
import pickle
import datetime
import time
import re
from multiprocessing import Process, Manager, Pool
import os, time, random
import pymongo 
from pymongo import MongoClient

/root/miniconda3/envs/hald/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [177]:
client = MongoClient(username="hald",password="XXX")

In [178]:
now=datetime.datetime.now()

In [6]:
now_time=now.strftime('%Y_%m_%d')

In [179]:
now_time

'2023_03_14'

now_time='2022_12_09'

In [180]:
pairs=['Gene','RNA','Carbohydrate','Lipid','Peptide', 'Pharmaceutical_Preparations', 'Protein', 'Toxin','Mutation', 'Disease']

In [181]:
db = client.hald

In [182]:
Gene_entities=list(db.Gene_raw.find({'addtime':now_time}))
RNA_entities=list(db.RNA_raw.find({'addtime':now_time}))
Carbohydrate_entities=list(db.Carbohydrate_raw.find({'addtime':now_time}))
Lipid_entities=list(db.Lipid_raw.find({'addtime':now_time}))
Peptide_entities=list(db.Peptide_raw.find({'addtime':now_time}))
Pharmaceutical_Preparations_entities=list(db.Pharmaceutical_Preparations_raw.find({'addtime':now_time}))
Protein_entities=list(db.Protein_raw.find({'addtime':now_time}))
Toxin_entities=list(db.Toxin_raw.find({'addtime':now_time}))


In [183]:
Mutation_entities=[]
Disease_entities=[]
for i in list(db.Mutation_raw.find({'addtime':now_time})):
    if i.get('Genomic View'):
        Mutation_entities.append(i)
for i in list(db.Disease_raw.find({'addtime':now_time})):
    if i.get('url'):
        Disease_entities.append(i)

In [184]:
names=locals()

In [185]:
for i in pairs:
    print(len(names[i+'_entities']))

2196
46
239
273
67
2
29
1
106
16323


In [186]:
for j in pairs:
    names[j+'_dict']={}
    for i in names[j+'_entities']:
        names[j+'_dict'][i.get('PMID')]=names[j+'_dict'].get(i.get('PMID'),[])+[i]


In [187]:
diff_relation_dict={}
for j in range(len(pairs)-1):
    for i in range(j+1,len(pairs)):
        for k in list(set(names[pairs[j]+'_dict'].keys()).intersection(set(names[pairs[i]+'_dict'].keys()))):
            for l in names[pairs[j]+'_dict'].get(k):
                for m in names[pairs[i]+'_dict'].get(k):
                    if l.get('sentence')==m.get('sentence') and l.get('entity')!=m.get('entity'):
                        diff_relation_dict[l.get('sentence')]=diff_relation_dict.get(l.get('sentence'),[])+[{'PMID':l.get('PMID'),
                                                                                                  'entity1':l.get('entity'),
                                                                                                   'entity2':m.get('entity'),
                                                                                                   'sentence':l.get('sentence'),
                                                                                                   'AB':l.get('AB'),
                                                                                                   'target1':l.get('target'),
                                                                                                    'target1_type':l.get('type'),
                                                                                                   'target2':m.get('target'),
                                                                                                    'target2_type':m.get('type'),
                                                                                                   'PMID':l.get('PMID'),
                                                                                                  }]

In [188]:
same_relation_dict={}
for j in range(len(pairs)):
    for k in names[pairs[j]+'_dict'].values():
        for l in range(len(k)-1):
            for m in range(l+1,len(k)):
                if k[l].get('sentence')==k[m].get('sentence'):
                    if k[l].get('entity')!=k[m].get('entity'):
                        same_relation_dict[k[l].get('sentence')]=same_relation_dict.get(k[l].get('sentence'),[])+[{'PMID':k[l].get('PMID'),
                                                                                                  'entity1':k[l].get('entity'),
                                                                                                   'entity2':k[m].get('entity'),
                                                                                                   'sentence':k[l].get('sentence'),
                                                                                                   'AB':k[l].get('AB'),
                                                                                                   'target1':k[l].get('target'),
                                                                                                    'target1_type':k[l].get('type'),
                                                                                                   'target2':k[m].get('target'),
                                                                                                    'target2_type':k[l].get('type'),
                                                                                                   'PMID':k[l].get('PMID'),
                                                                                                  }]

In [189]:
print(len(same_relation_dict))
print(len(diff_relation_dict))
print(len(list(set(list(same_relation_dict.keys())).intersection(set(list(diff_relation_dict.keys()))))))

3188
731
271


In [190]:
relation_dict={}
for i in same_relation_dict.keys():
    if diff_relation_dict.get(i):
        relation_dict[i]=same_relation_dict.get(i)+diff_relation_dict.get(i)

In [191]:
for i in same_relation_dict.keys():
    if not relation_dict.get(i):
        relation_dict[i]=same_relation_dict.get(i)
print(len(relation_dict))
for i in diff_relation_dict.keys():
    if not relation_dict.get(i):
        relation_dict[i]=diff_relation_dict.get(i)
print(len(relation_dict))

3188
3648


In [26]:
import os
os.makedirs('results/step3/openie/filelist/')

In [27]:
ind=1
for i in relation_dict.keys():
    f=open('results/step3/openie/filelist/sentence'+str(ind)+'.txt','w')
    f.write(i)
    f.close()
    ind+=1

In [28]:

f=open('results/step3/openie/filelist.txt','a')
for i in range(1,len(relation_dict)+1):
    f.write('filelist/sentence'+str(i)+'.txt'+'\n')
f.close()

In [29]:
len(relation_dict)

3648

In [30]:
sentences=list(relation_dict.keys())

In [31]:
os.makedirs('results/step3/openie/files/')
k=1
for i in range(0,len(sentences),len(sentences)//20):
    pickle.dump(sentences[i:i+len(sentences)//20],open('results/step3/openie/files/openie'+str(k)+'.pkl','wb'))
    k+=1

In [32]:
os.makedirs('results/step3/rules/files/')
k=1
for i in range(0,len(sentences),len(sentences)//20):
    f=open('results/step3/rules/files/rules'+str(k)+'.py','w')
    f.write('''
import stanza
import os
import datetime
import time
nlp = stanza.Pipeline('en', package='genia',download_method=None)
import pickle
raw=pickle.load(open('../../openie/files/openie'''+str(k)+'''.pkl','rb'))
import re 

def nlpSentence(i,sentence2words,sentence2relations,sentence2verblist,sentence2verbtype,sentence2adv):
    sentence=i
    doc=nlp(sentence)
    words=[]
    for sent in doc.sentences:
        for word in sent.words:
            words.append(word.text)
    relations=[]
    verblist=[]
    verbprototype=[]
    adv=[]
    for sent in doc.sentences:
        for word in sent.words:
            relations.append([word.text,words[int(word.head)-1],int(word.id),int(word.head),word.deprel,word.xpos])
            if word.xpos in ['VB','VBD','VBG','VBN', 'VBP', 'VBZ']:
                verblist.append({word.text:word.xpos})
                verbprototype.append({word.text:word.lemma})
            if word.xpos in ['RB','RBR','RBS']:
                adv.append({word.text:word.xpos})
    sentence2words[i]=words
    sentence2relations[i]=relations
    sentence2verblist[i]=verblist
    sentence2verbprototype[i]=verbprototype
    sentence2adv[i]=adv


start=datetime.datetime.now()
print('Parent process %s.' % os.getpid())
sentence2words={}
sentence2relations={}
sentence2verblist={}
sentence2verbprototype={}
sentence2adv={}
for i in raw:
    nlpSentence(i,sentence2words,sentence2relations,sentence2verblist,sentence2verbprototype,sentence2adv)
pickle.dump(sentence2words,open('sentence2words'''+str(k)+'''.pkl','wb'))
pickle.dump(sentence2relations,open('sentence2relations'''+str(k)+'''.pkl','wb'))
pickle.dump(sentence2verblist,open('sentence2verblist'''+str(k)+'''.pkl','wb'))
pickle.dump(sentence2verbprototype,open('sentence2verbprototype'''+str(k)+'''.pkl','wb'))
pickle.dump(sentence2adv,open('sentence2adv'''+str(k)+'''.pkl','wb'))
print('Waiting for all subprocesses done...')

end=datetime.datetime.now()
print("The running time is "+str((end-start).seconds)+"s")  
    ''')
    f.close()
    k+=1

In [33]:
pwd

'/root/hald_autoupdate/2023_03_13'

In [ ]:
# shell
# cd openie
# nohup java -mx500g -cp ../../../../stanza_corenlp/stanford-corenlp-4.3.2.jar:../../../../stanza_corenlp/stanford-corenlp-4.3.2-models.jar:../../../../stanza_corenlp/CoreNLP-to-HTML.xsl:../../../../slf4j-api.jar:../../../../stanza_corenlp/slf4j-simple.jar edu.stanford.nlp.naturalli.OpenIE -format reverb -filelist ./filelist.txt -output ./openie.txt > ./openie.log &

In [46]:
#shell 
# cd rules/files
# for i in {1..21}; do echo "nohup python rules${i}.py > rules.log &" | bash; done

In [34]:
import pandas as pd

In [36]:
openie_table=pd.read_table('results/step3/openie/openie.txt',header=None)

In [173]:
openie_table.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,filelist/sentence19.txt,0,DDIT3 knockdown,is,able,2,4,13,14,14,15,1.0,"Moreover , DDIT3 knockdown in CD34 + cells fro...","RB , NN NN IN NN SYM NNS IN NNP NNS IN NN VBZ ...",ddit3 knockdown,be,able
1,filelist/sentence19.txt,0,DDIT3 knockdown,is in,CD34 cells,2,4,4,5,5,8,1.0,"Moreover , DDIT3 knockdown in CD34 + cells fro...","RB , NN NN IN NN SYM NNS IN NNP NNS IN NN VBZ ...",ddit3 knockdown,be in,cd34 cell
2,filelist/sentence19.txt,0,MDS patients,is with,anemia,9,11,11,12,12,13,1.0,"Moreover , DDIT3 knockdown in CD34 + cells fro...","RB , NN NN IN NN SYM NNS IN NNP NNS IN NN VBZ ...",MDS patient,be with,anemia
3,filelist/sentence19.txt,0,DDIT3 knockdown,restore,erythropoiesis,2,4,16,17,17,18,1.0,"Moreover , DDIT3 knockdown in CD34 + cells fro...","RB , NN NN IN NN SYM NNS IN NNP NNS IN NN VBZ ...",ddit3 knockdown,restore,erythropoiesis
4,filelist/sentence19.txt,0,DDIT3 knockdown,is,Moreover able,2,4,13,14,0,15,1.0,"Moreover , DDIT3 knockdown in CD34 + cells fro...","RB , NN NN IN NN SYM NNS IN NNP NNS IN NN VBZ ...",ddit3 knockdown,be,moreover able


In [38]:
# 3770378
len(openie_table)

54068

In [39]:
df=openie_table.loc[:,[0,2,3,4,12]]

In [40]:
sentences=list(relation_dict.keys())

In [41]:
be=['be','am','is','are','was','were','have been','has been']
bedict={}
for i in be:
    bedict[i]='be'


In [42]:
rules_pairs=['sentence2adv','sentence2relations','sentence2verblist','sentence2verbprototype','sentence2words']

In [43]:
pwd

'/root/hald_autoupdate/2023_03_13'

In [44]:
for i in rules_pairs:
    names[i]={}
    for k in ['results/step3/rules/files/'+i+str(j)+'.pkl' for j in range(1,22)]:
        names[i].update(pickle.load(open(k,'rb')))
    # pickle.dump(names[i],open('files/'+i+'.pkl','wb'))

In [218]:
finallist=[]
for line in range(len(df)):
    i=sentences[int(df.iloc[line,0][17:-4])-1]
    verblist=sentence2verblist.get(i)
    verbrelation=''
    real_verb=''
    verbdict={}
    verbs_split=df.loc[line,3].split(' ')
    for verb in verblist:
        verbdict.update(verb)
    # if len(verbs.split(' '))==1:
    #     real_verb=verbs.split(' ')[0]
    # else:

    for indexx,one in enumerate(verbs_split):
        if verbdict.get(one):
            verbrelation=one
            relations=sentence2relations.get(i)
            relations_display=[relation[-2] for relation in relations]
            verbprototype=sentence2verbprototype.get(i)
            verbprototype_dict={}
            for verbp in verbprototype:
                verbprototype_dict.update(verbp)
            for index_passive,passive in enumerate(relations_display):
                if passive=='aux:pass':
                    if relations[index_passive][1]==verbrelation:
                        verb_prototype=verbrelation
                        break
                elif passive=='acl' and relations[index_passive][-1]=='VBN':
                    if relations[index_passive][0]==verbrelation:
                        verb_prototype=verbrelation
                        break
            else:
                for v in verbprototype:
                    if v.get(verbrelation):
                        verb_prototype=v.get(verbrelation)
                        break
            verbs_split[indexx]=verb_prototype
        elif bedict.get(one):
            verbs_split[indexx]=bedict.get(one)
    if bedict.get(' '.join(verbs_split[:2])):
        final=' '.join(['be']+verbs_split[2:])
    else:
        final=' '.join(verbs_split)
    finallist.append(final)
df[17]=finallist

In [46]:
ind=1
for i in relation_dict.keys():
    names['sentence'+str(ind)]=i
    ind+=1

In [47]:
import copy

In [48]:
relation_dict_copy=copy.deepcopy(relation_dict)

In [219]:
import re
suc=[]
for i in range(1,len(relation_dict_copy)+1):
    if relation_dict_copy.get(names[df.iloc[i-1,0][9:-4]]):
        openie=[]
        for j in relation_dict_copy.get(names[df.iloc[i-1,0][9:-4]]):
            j1=j.get('target1').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
            j2=j.get('target2').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
            if re.search(j1,df.iloc[i-1,1]) and re.search(j2,df.iloc[i-1,3]):
                j['openie']=j.get('openie',[])+[[j.get('target1'),df.iloc[i-1,-1],j.get('target2')]]
                openie.append([j.get('target1'),df.iloc[i-1,-1],j.get('target2')])
            elif re.search(j2,df.iloc[i-1,1]) and re.search(j1,df.iloc[i-1,3]):
                j['openie']=j.get('openie',[])+[[j.get('target2'),df.iloc[i-1,-1],j.get('target1')]]
                openie.append([j.get('target2'),df.iloc[i-1,-1],j.get('target1')]) 
        if not openie:
            suc.append(names[df.iloc[i-1,0][9:-4]])
    
        

In [220]:
error=[]
for i in range(1,len(relation_dict_copy)+1):
    if not relation_dict_copy.get(names[df.iloc[i-1,0][9:-4]]):
        error.append(names[df.iloc[i-1,0][9:-4]])

In [221]:
len(suc)

3327

In [222]:
len(relation_dict_copy)

3648

In [53]:
len(error)

0

In [54]:
import sys
sys.path.append("..")
from allennlp.predictors.predictor import Predictor
import allennlp_models.structured_prediction
PREDICTOR = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz")

In [55]:
import re

def clean(sent):
    return re.sub(' +', ' ', sent.replace('\r', ' ').replace('\n', ' '))


def get_tags_allen(sent):
    sent = sent.replace('[', '(').replace(']', ')')
    res = PREDICTOR.predict(
      sentence=sent
    )
    return res


def parse_allen(ss):
    words = ss['words']
    
    res = []
    for v in ss['verbs']:
        tags = v['tags']

        d = {}
        for i in range(len(words)):
            t = tags[i].replace('I-', '').replace('B-', '')
            w = words[i]
            if t in d:
                d[t] += ' '+ w
            else:
                d[t] = w
        res.append(d)
    return res

In [ ]:
sentence2allennlp={}
for i in relation_dict_copy.keys():
    res=get_tags_allen(i)
    ress=parse_allen(res)
    sentence2allennlp[i]=ress

In [ ]:
import re
for i,j in relation_dict_copy.items():
    for k in j:
        k1=k.get('target1').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                  "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
        k2=k.get('target2').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                  "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
        for l in sentence2allennlp.get(i):
            if l.get('ARG1') and l.get('ARG2') and l.get('V'):
                if re.search(k1,l.get('ARG1')) and re.search(k2,l.get('ARG2')):
                    k['allennlp']=k.get('allennlp',[])+[[k.get('target1'), l.get('V') ,k.get('target2')]]
                elif re.search(k2,l.get('ARG1')) and re.search(k1,l.get('ARG2')):
                    k['allennlp']=k.get('allennlp',[])+[[k.get('target2'), l.get('V') ,k.get('target1')]]

In [ ]:
for i,j in relation_dict_copy.items():
    for k in j:
        if k.get('allennlp'):
            print(k)
            break
    else:
        continue
    break

In [ ]:
len(sentence2adv)

In [ ]:
pwd

In [61]:
pwd

'/root/hald_autoupdate/2023_03_13'

In [62]:
negativewords=['no','not','never','hardly','barely','scarcely','rarely','few','little','seldom','neither','nor','negatively']
negativewords_dict={}
for i in negativewords:
    negativewords_dict[i]=i

for i,j in relation_dict_copy.items():
    for k in j:
        if k.get('openie'):
            del k['openie']

In [223]:
import datetime
import networkx as nx
import matplotlib.pyplot as plt
from pylab import mpl
import re
from multiprocessing import Process, Manager, Pool
import os, time, random
def work(i,j,triple,abnormal, negative_sentences,networkx_raw):
    adv=sentence2adv.get(i)
    words=sentence2words.get(i)

    sentence=i
    # length=len(sentence2words.get(i))

    relations=sentence2relations.get(i)
    relations_display=[relation[-2] for relation in relations]
    verblist=sentence2verblist.get(i)
    verbprototype=sentence2verbprototype.get(i)
    verbprototype_dict={}
    for verbp in verbprototype:
        verbprototype_dict.update(verbp)
    # for k in range(length):
    G = nx.Graph()  # 建立无向图G

    # 添加节点
    for word in words:
        G.add_node(word)

    G.add_node('root')

    # 添加边
    for r in range(len(relations)):
        if relations[r][4]=='root':
            G.add_edge(relations[r][0], 'root',labels=relations[r][4])
        else:
            G.add_edge(relations[r][0], relations[r][1],labels=relations[r][4])

    for k in j:
        if verblist:
            k1=k.get('target1').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
            k2=k.get('target2').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
            if re.search(k1,sentence) and re.search(k2,sentence):
                worddict={}
                for word in words:
                    worddict[word]=word
                if re.search(k1,sentence).span()[0] < re.search(k2,sentence).span()[0]:
                    k1_word=k.get('target1')
                    k2_word=k.get('target2')
                    if not worddict.get(k1_word):
                        for word in words:
                            if re.search(r'-',word) and re.search(k1,word):
                                k1_word=word
                    if not worddict.get(k2_word):
                        for word in words:
                            if re.search(r'-',word) and re.search(k2,word):
                                k2_word=word
                    try:
                        path=nx.shortest_path(G, source=re.split('[ -]',k1_word)[-1], target=re.split('[ -]',k2_word)[-1])
                    except:
                        abnormal.append(i)
                    else:
                        verbdict={}
                        for verb in verblist:
                            verbdict.update(verb)
                        verbrelation=''
                        for one in path:
                            if verbdict.get(one):
                                verbrelation=one

                        # verbprototype[verblist.index(verb)]    
                        if verbrelation :
                            negative=False
                            for index_passive,passive in enumerate(relations_display):
                                if passive=='aux:pass':
                                    if relations[index_passive][1]==verbrelation:
                                        verb_prototype=verbrelation
                                        break
                                elif passive=='acl' and relations[index_passive][-1]=='VBN':
                                    if relations[index_passive][0]==verbrelation:
                                        verb_prototype=verbrelation
                                        break
                            else:
                                for v in verbprototype:
                                    if v.get(verbrelation):
                                        verb_prototype=v.get(verbrelation)
                                        break
                            if verb_prototype:
                                for relation in relations:
                                    if relation[1]==verbrelation and negativewords_dict.get(relation[0]):
                                        negative=True
                                if not negative:
                                    triple.append({'triple':[k.get('target1'),verb_prototype,k.get('target2')],\
                                                        'PMID':k.get('PMID'), 'sentence':sentence,'AB':k.get('AB'),
                                                   'former_target':k.get('target1'),'former_entity':k.get('entity1'),'former_entity_type':k.get('target1_type'),
                                                   'verb':verbrelation,'verbprototype':verbprototype_dict.get(verbrelation),'latter_target':k.get('target2'),
                                                   'latter_entity':k.get('entity2'),'latter_entity_type':k.get('target2_type')})
                                    k['networkx']=[k.get('target1'),verb_prototype,k.get('target2')]
                                else:
                                    negative_sentences.append(i)
                else:
                    k1_word=k.get('target1')
                    k2_word=k.get('target2')
                    if not worddict.get(k1_word):
                        for word in words:
                            if re.search(r'-',word) and re.search(k1,word):
                                k1_word=word
                    if not worddict.get(k2_word):
                        for word in words:
                            if re.search(r'-',word) and re.search(k2,word):
                                k2_word=word
                    try:
                        path=nx.shortest_path(G, source=re.split('[ -]',k2_word)[-1], target=re.split('[ -]',k1_word)[-1])
                    except:
                        abnormal.append(i)
                    else:
                        verbdict={}
                        for verb in verblist:
                            verbdict.update(verb)
                        verbrelation=''
                        for one in path:
                            if verbdict.get(one):
                                verbrelation=one
                        # verbprototype[verblist.index(verb)]    
                        if verbrelation:
                            negative=False
                            for index_passive,passive in enumerate(relations_display):
                                if passive=='aux:pass':
                                    if relations[index_passive][1]==verbrelation:
                                        verb_prototype=verbrelation
                                        break
                                elif passive=='acl' and relations[index_passive][-1]=='VBN':
                                    if relations[index_passive][0]==verbrelation:
                                        verb_prototype=verbrelation
                                        break
                            else:
                                for v in verbprototype:
                                    if v.get(verbrelation):
                                        verb_prototype=v.get(verbrelation)
                                        break
                            if verb_prototype:
                                for relation in relations:
                                    if relation[1]==verbrelation and negativewords_dict.get(relation[0]):
                                        negative=True
                                if not negative:
                                    triple.append({'triple':[k.get('target2'),verb_prototype,k.get('target1')],\
                                                        'PMID':k.get('PMID'), 'sentence':sentence,'AB':k.get('AB'),
                                                   'former_target':k.get('target2'),'former_entity':k.get('entity2'),'former_entity_type':k.get('target2_type'),
                                                   'verb':verbrelation,'verbprototype':verbprototype_dict.get(verbrelation),'latter_target':k.get('target1'),
                                                   'latter_entity':k.get('entity1'),'latter_entity_type':k.get('target1_type')})
                                    k['networkx']=[k.get('target2'),verb_prototype,k.get('target1')]
                                else:
                                    negative_sentences.append(i)
        networkx_raw.append(k)
if __name__=='__main__':
    start=datetime.datetime.now()
    print('Parent process %s.' % os.getpid())
    p=Pool(40)
    manager=Manager()
    triple=manager.list()
    abnormal=manager.list()
    negative_sentences=manager.list()
    networkx_raw=manager.list()
    for i,j in relation_dict_copy.items():
        p.apply_async(work, args=(i,j,triple,abnormal,negative_sentences,networkx_raw))

    print('Waiting for all subprocesses done...')
    
    p.close()
    p.join()
    print('All subprocesses done.')

    end=datetime.datetime.now()
    print("The running time is "+str((end-start).seconds)+"s")  

Parent process 24627.
Waiting for all subprocesses done...
All subprocesses done.
The running time is 31s


In [224]:
triple_list=list(triple)
abnormal_list=list(abnormal)
negative_list=list(negative_sentences)
networkx_raw_list=list(networkx_raw)
networkx_raw_dict={}
for i in networkx_raw_list:
    networkx_raw_dict[i.get('sentence')]=networkx_raw_dict.get(i.get('sentence'),[])+[i]

In [225]:
pickle.dump(triple_list,open('results/step3/triple_list.pkl','wb'))

In [226]:
pickle.dump(abnormal_list,open('results/step3/abnormal_list.pkl','wb'))
pickle.dump(negative_list,open('results/step3/negative_list.pkl','wb'))
pickle.dump(networkx_raw_list,open('results/step3/networkx_raw_list.pkl','wb'))
pickle.dump(networkx_raw_dict,open('results/step3/networkx_raw_dict.pkl','wb'))

In [227]:
raw_pick_pubtator=pickle.load(open('results/step1/raw_pick_pubtator.pkl','rb'))
rawpickwithAB={}
for i in raw_pick_pubtator:
    rawpickwithAB[i.get('PMID')]=i

In [228]:
networkx_raw_list[0]

{'PMID': '36845012',
 'entity1': 'IER3',
 'entity2': 'ARNTL',
 'sentence': 'Among the differentially expressed genes, aryl hydrocarbon receptor nuclear translocator-like (ARNTL), BTG antiproliferation factor 2 (BTG2), C-X-C motif chemokine ligand 10 (CXCL10), chitinase 3-like 1 (CHI3L1), immediate early response 3 (IER3), Fos proto-oncogene, AP-1 transcription factor subunit (FOS), and peroxisome proliferative activated receptor, gamma, coactivator 1 alpha (PPARGC1A), mainly involved in the regulation of cell proliferation, metabolism, and inflammation, were also dysregulated in liver tissues suffered from IRI and could form a FOS-centered interaction network.',
 'AB': 'Background: With the intensification of population aging, the proportion of aging livers in the donor pool is increasing rapidly. Compared with young livers, aging livers are more susceptible to ischemia-reperfusion injury (IRI) during liver transplantation, which greatly affects the utilization rate of aging livers. Th

In [229]:
len(networkx_raw_dict)

3648

In [230]:
be=['be','am','is','are','was','were','have been','has been']
bedict={}
for i in be:
    bedict[i]=i
can=['can','could','will','would','may','might']
candict={}
for i in can:
    candict[i]=i

In [231]:
for i in networkx_raw_list:
    openie_mid=[]
    openie_final=[]
    if i.get('openie'):
        for j in i.get('openie'):
            if not bedict.get(j[1]) and not candict.get(j[1]):
                openie_mid.append(j)
        adv=sentence2adv.get(i.get('sentence'))
        adv_dict={}
        for l in adv:
            adv_dict.update(l)
        for k in openie_mid:
            for m in k[1].split(' '):
                if adv_dict.get(m):
                    break
            else:
                if k not in openie_final:
                    openie_final.append(k)
    i['openie_final']=openie_final
        

In [232]:
for i in networkx_raw_list:
    allennlp_final=[]
    if i.get('allennlp'):
        for j in i.get('allennlp'):
            if not bedict.get(j[1]) and not candict.get(j[1]):
                allennlp_final.append(j)
    i['allennlp_final']=allennlp_final

In [233]:
for i in networkx_raw_list:
    if i.get('networkx'):
        if bedict.get(i.get('networkx')[1]):
            i['networkx']=''
        elif i.get('networkx')[1]=='"':
            i['networkx']=''
    else:
        i['networkx']=''

In [234]:
networkx_raw_final=[]
for i in networkx_raw_list:
    if i.get('networkx') or i.get('openie_final') or i.get('allennlp_final'):
        networkx_raw_final.append(i)

In [235]:
len(networkx_raw_final)

2042

In [236]:
for i in networkx_raw_final:
    bidirection=False
    if i.get('networkx'):
        if i.get('networkx')[1]=='associated' or i.get('networkx')[1]=='associate' :
            i['bidirection']=True
            continue
    if i.get('allennlp'):
        for j in i.get('openie_final'):
            if j=='associated' or j=='associate' :
                bidirection=True
                break
    if i.get('openie_final'):
        for j in i.get('openie_final'):
            for k in j[1].split(' '):
                if k=='associated' or  k=='associate' :
                    bidirection=True
                    break
            else:
                continue
            break
    i['bidirection']=bidirection
    

In [237]:
import copy

In [238]:
Relation_raw=copy.deepcopy(networkx_raw_final)

In [239]:
pickle.dump(networkx_raw_final,open('results/step3/networkx_raw_final.pkl','wb'))

In [240]:
networkx_raw_final[0]

{'PMID': '36071684',
 'entity1': 'ERCC4',
 'entity2': 'Atherosclerosis',
 'sentence': 'Taken together, these findings indicated that lncRNA_FGF7-5 and lncRNA_GLRX3 together reduced atherosclerosis-induced apoptosis of HUVECs via targeting miR-2681-5p to increase ERCC4 expression, thereby preventing the formation of carotid plaque and finally inhibiting atherosclerosis progression.',
 'AB': 'Atherosclerotic plaques belong to the common vascular disease in the aged, which rupture will lead to acute thromboembolic diseases, the leading cause of fatal cardiovascular events. Accumulating evidence indicates that the lncRNAs-miRNAs-mRNA regulatory network plays a critical role in atherosclerosis. Based on RNA sequencing (GSE207252), we constructed expression profiles of lncRNAs, microRNAs, and mRNA in the carotid plaque of atherosclerosis patients and analyzed differentially expressed genes (DEGs). We identified three candidate lncRNAs using two algorithms (LASSO and SVM-RFE): lnc_GLRX3, lnc_

In [241]:
for i in networkx_raw_final:
    if i.get('openie_final') and i.get('openie_final')[0][1]=='highlights':
        print(i)
        break

In [242]:
# networkx_raw_final=pickle.load(open('networkx_raw_final.pkl','rb'))

In [243]:
len(networkx_raw_final)

2042

In [244]:
networkx_raw_final[0]

{'PMID': '36071684',
 'entity1': 'ERCC4',
 'entity2': 'Atherosclerosis',
 'sentence': 'Taken together, these findings indicated that lncRNA_FGF7-5 and lncRNA_GLRX3 together reduced atherosclerosis-induced apoptosis of HUVECs via targeting miR-2681-5p to increase ERCC4 expression, thereby preventing the formation of carotid plaque and finally inhibiting atherosclerosis progression.',
 'AB': 'Atherosclerotic plaques belong to the common vascular disease in the aged, which rupture will lead to acute thromboembolic diseases, the leading cause of fatal cardiovascular events. Accumulating evidence indicates that the lncRNAs-miRNAs-mRNA regulatory network plays a critical role in atherosclerosis. Based on RNA sequencing (GSE207252), we constructed expression profiles of lncRNAs, microRNAs, and mRNA in the carotid plaque of atherosclerosis patients and analyzed differentially expressed genes (DEGs). We identified three candidate lncRNAs using two algorithms (LASSO and SVM-RFE): lnc_GLRX3, lnc_

In [245]:
collections_Relation_raw=db.Relation_raw

In [246]:
now_time

'2023_03_14'

collections_Relation_raw.remove({'addtime':now_time})

In [248]:
for i in Relation_raw:
    i['addtime']=now_time

In [249]:
collections_Relation_raw.insert_many(Relation_raw)

In [250]:
Relation_raw_all_with_disease=copy.deepcopy(Relation_raw)

In [251]:
len(Relation_raw_all_with_disease)

2042

In [252]:
Relation_raw_all_with_disease[10]

{'PMID': '36378468',
 'entity1': 'MSI2',
 'entity2': 'Memory Disorders',
 'sentence': 'Furthermore, pharmacological inhibition of human MSI1 and MSI2 activity using (-)- gossypol resulted in improved memory retention, without causing locomotor, chemotactic, or learning deficits.',
 'AB': "Musashi RNA-binding proteins (MSIs) retain a pivotal role in stem cell maintenance, tumorigenesis, and nervous system development. Recently, we showed in C. elegans that Musashi (MSI-1) actively promotes forgetting upon associative learning via a 3'UTR-dependent translational expression of the Arp2/3 actin branching complex. Here, we investigated the evolutionary conserved role of MSI proteins and the effect of their pharmacological inhibition on memory. Expression of human Musashi 1 (MSI1) and Musashi 2 (MSI2) under the endogenous Musashi promoter fully rescued the phenotype of msi-1(lf) worms. Furthermore, pharmacological inhibition of human MSI1 and MSI2 activity using (-)- gossypol resulted in imp

In [253]:
all_es=json.load(open('results/step2/all_es_lack.json'))

In [254]:
# multi_cheicals=[]
multi_chemical_type2={}
for i in all_es:
    if not multi_chemical_type2.get(i.get('entity')):
        multi_chemical_type2[i.get('entity')]=[i.get('type')]
    else:
        if i.get('type') not in multi_chemical_type2.get(i.get('entity')):
            multi_chemical_type2[i.get('entity')]+=[i.get('type')]

In [255]:
cxz=0
for i,j in multi_chemical_type2.items():
    if len(j)>1:
        cxz+=1
        print({i:j})
print(cxz)

{'Lipopolysaccharides': ['Carbohydrate', 'Lipid', 'Toxin']}
{'Vancomycin': ['Carbohydrate', 'Peptide']}
{'Ceramides': ['Carbohydrate', 'Lipid']}
{'Sphingomyelins': ['Carbohydrate', 'Lipid']}
{'Bleomycin': ['Carbohydrate', 'Peptide']}
{'Teicoplanin': ['Carbohydrate', 'Peptide']}
{'Lipid A': ['Carbohydrate', 'Lipid', 'Toxin']}
{'G(M1) Ganglioside': ['Carbohydrate', 'Lipid']}
{'Gangliosides': ['Carbohydrate', 'Lipid']}
{'Glycosphingolipids': ['Carbohydrate', 'Lipid']}
{'Sulfoglycosphingolipids': ['Carbohydrate', 'Lipid']}
{'Glycolipids': ['Carbohydrate', 'Lipid']}
{'G(M3) Ganglioside': ['Carbohydrate', 'Lipid']}
{'Glycopeptides': ['Carbohydrate', 'Peptide']}
{'Glycosylphosphatidylinositols': ['Carbohydrate', 'Lipid']}
{'Globosides': ['Carbohydrate', 'Lipid']}
{'Ristocetin': ['Carbohydrate', 'Peptide']}
{'Glucosylceramides': ['Carbohydrate', 'Lipid']}
{'Holothurin': ['Carbohydrate', 'Toxin']}
{'Glycerylphosphorylcholine': ['Carbohydrate', 'Lipid']}
{'Glycerophosphates': ['Carbohydrate', 'L

In [256]:
import copy
Relation_raw_all_final=[]
for i in Relation_raw_all_with_disease:
    if multi_chemical_type2.get(i.get('entity1')) and multi_chemical_type2.get(i.get('entity2')):
        for j in multi_chemical_type2.get(i.get('entity1')):
            for k in multi_chemical_type2.get(i.get('entity2')):
                mid=copy.deepcopy(i)
                mid.update({'target1_type':j,'target2_type':k})
                Relation_raw_all_final.append(mid)
    else:
        Relation_raw_all_final.append(i)
        if multi_chemical_type2.get(i.get('entity1')):
            for j in multi_chemical_type2.get(i.get('entity1')):
                if i.get('target1_type') != j:
                    mid=copy.deepcopy(i)
                    mid.update({'target1_type':j})
                    Relation_raw_all_final.append(mid)
        elif multi_chemical_type2.get(i.get('entity2')):
            for j in multi_chemical_type2.get(i.get('entity2')):
                if i.get('target2_type') != j:
                    mid=copy.deepcopy(i)
                    mid.update({'target2_type':j})
                    Relation_raw_all_final.append(mid)

In [257]:
#122534
len(Relation_raw_all_final)

2074

In [258]:
direction=set()
direction_list=[]
direction_type=[]
for i in Relation_raw_all_final:

    if i.get('openie_final'):
        if i.get('openie_final')[0][0]!=i.get('target1'):
            i['entity2'],i['entity1']=i['entity1'],i['entity2']
            i['target2_type'],i['target1_type']=i['target1_type'],i['target2_type']
            i['target2'],i['target1']=i['target1'],i['target2']
    elif i.get('networkx'):
        if i.get('networkx')[0]!= i.get('target1'):
            i['entity2'],i['entity1']=i['entity1'],i['entity2']
            i['target2_type'],i['target1_type']=i['target1_type'],i['target2_type']
            i['target2'],i['target1']=i['target1'],i['target2']
    elif i.get('allennlp_final'):
        if i.get('allennlp_final')[0][0]!=i.get('target1'):
            i['entity2'],i['entity1']=i['entity1'],i['entity2']
            i['target2_type'],i['target1_type']=i['target1_type'],i['target2_type']
            i['target2'],i['target1']=i['target1'],i['target2']
    if i.get('bidirection'):
        if (i.get('entity2'),(i.get('entity1'))) not in direction :
            direction.add((i.get('entity2'),(i.get('entity1'))))
            direction_list.append((i.get('entity2'),(i.get('entity1'))))
            direction_type.append((i.get('target2_type'),(i.get('target1_type'))))
        if (i.get('entity1'),(i.get('entity2'))) not in direction:
            direction.add((i.get('entity1'),(i.get('entity2'))))
            direction_list.append((i.get('entity1'),(i.get('entity2'))))
            direction_type.append((i.get('target1_type'),(i.get('target2_type'))))
    else:
        direction.add((i.get('entity1'),(i.get('entity2'))))
        direction_list.append((i.get('entity1'),(i.get('entity2'))))
        direction_type.append((i.get('target1_type'),(i.get('target2_type'))))

In [259]:
Relation_raw_all_final[10]

{'PMID': '36378468',
 'entity1': 'MSI2',
 'entity2': 'Memory Disorders',
 'sentence': 'Furthermore, pharmacological inhibition of human MSI1 and MSI2 activity using (-)- gossypol resulted in improved memory retention, without causing locomotor, chemotactic, or learning deficits.',
 'AB': "Musashi RNA-binding proteins (MSIs) retain a pivotal role in stem cell maintenance, tumorigenesis, and nervous system development. Recently, we showed in C. elegans that Musashi (MSI-1) actively promotes forgetting upon associative learning via a 3'UTR-dependent translational expression of the Arp2/3 actin branching complex. Here, we investigated the evolutionary conserved role of MSI proteins and the effect of their pharmacological inhibition on memory. Expression of human Musashi 1 (MSI1) and Musashi 2 (MSI2) under the endogenous Musashi promoter fully rescued the phenotype of msi-1(lf) worms. Furthermore, pharmacological inhibition of human MSI1 and MSI2 activity using (-)- gossypol resulted in imp

In [260]:
pairs

['Gene',
 'RNA',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 'Protein',
 'Toxin',
 'Mutation',
 'Disease']

In [261]:
pwd

'/root/hald_autoupdate/2023_03_13'

In [262]:
rules_pairs=['sentence2adv','sentence2relations','sentence2verblist','sentence2verbprototype','sentence2words']
names=locals()
for i in rules_pairs:
    names[i]={}
    for k in ['results/step3/rules/files/'+i+str(j)+'.pkl' for j in range(1,22)]:
        names[i].update(pickle.load(open(k,'rb')))

In [344]:
import copy
Relation_raw_all_final_copy=copy.deepcopy(Relation_raw_all_final)

In [345]:
be=['be','am','is','are','was','were','have been','has been']
bedict={}
for i in be:
    bedict[i]=i
can=['can','could','will','would','may','might']
candict={}
for i in can:
    candict[i]=i
countt=0
counttt=0
for i in Relation_raw_all_final_copy:
    verblist=sentence2verblist.get(i.get('sentence'))
    verbrelation=''
    real_verb=''
    verbdict={}
    for verb in verblist:
        verbdict.update(verb)
    for indexx,j in enumerate(i.get('allennlp_final')):
        one=j[1]
        if verbdict.get(one):
            verbrelation=one
            relations=sentence2relations.get(i.get('sentence'))
            relations_display=[relation[-2] for relation in relations]
            verbprototype=sentence2verbprototype.get(i.get('sentence'))
            verbprototype_dict={}
            for verbp in verbprototype:
                verbprototype_dict.update(verbp)
            for index_passive,passive in enumerate(relations_display):
                if passive=='aux:pass':
                    if relations[index_passive][1]==verbrelation:
                        verb_prototype=verbrelation

                        break
                elif passive=='acl' and relations[index_passive][-1]=='VBN':
                    if relations[index_passive][0]==verbrelation:
                        verb_prototype=verbrelation

                        break
            else:
                for v in verbprototype:
                    if v.get(verbrelation):
                        verb_prototype=v.get(verbrelation)
                        break
            if verb_prototype!=one:
                if countt<10:
                    countt+=1
                    # print(i.get('allennlp_final'))
                    # print(i.get('sentence'))
                    # print(verb_prototype)
            i['allennlp_final'][indexx][1]=verb_prototype
        elif bedict.get(one):
            i['allennlp_final'][indexx][1]=bedict.get(one)



drop_all1=[]
drop_all2=[]
for i in Relation_raw_all_final_copy:
    sentence2verbdict={}
    for j in sentence2verblist.get(i.get('sentence')):
        sentence2verbdict[list(j.keys())[0]]=list(j.keys())[0]
    for j in sentence2verbprototype.get(i.get('sentence')):
        sentence2verbdict[list(j.values())[0]]=list(j.values())[0]
    sentence2verbdict.update(bedict)
    if i.get('openie_final'):
        openie_final=[]
        for j in i.get('openie_final'):
            openi=False
            for k in j[1].split(' '):
                if sentence2verbdict.get(k):
                    openi=True
                    break
            if openi:
                openie_final.append(j)
            else:
                drop_all1.append(j)
        # if len(openie_final) != len(i.get('openie_final')):
        #     drop_all1.append(openie_final)
        #     drop_all2.append(i.get('openie_final'))
        i['openie_final']=openie_final
#     if i.get('allennlp_final'):
#         allennlp_final=[]
#         for j in i.get('allennlp_final'):
#             allennl=False
#             for k in j[1].split(' '):
#                 if sentence2verbdict.get(k):
#                     allennl=True
#                     break
#             if allennl:
#                 allennlp_final.append(j)
#             else:
#                 drop_all2.append(j)
#         # if len(allennlp_final) != len(i.get('allennlp_final')):
#         #     drop_all1.append(allennlp_final)
#         #     drop_all2.append(i.get('allennlp_final'))
#         i['allennlp_final']=allennlp_final

In [346]:
len(Relation_raw_all_final_copy)

2074

In [347]:
for i in Relation_raw_all_final_copy:
    if i.get('_id'):
        del i['_id']
json.dump(Relation_raw_all_final_copy,open('results/step3/triplelist.json','w'))

In [348]:
len(Relation_raw_all_final_copy)

2074

In [349]:
collections_Relation_final=db.Relation_final

collections_Relation_final.remove({'addtime':now_time})

In [350]:
for i in Relation_raw_all_final_copy:
    i['addtime']=now_time

In [351]:
collections_Relation_final.insert_many(Relation_raw_all_final_copy)

In [352]:
pickle.dump(Relation_raw_all_final_copy,open('results/step3/Relation_raw_all_final_copy.pkl','wb'))

In [353]:
pickle.dump(Relation_raw,open('results/step3/Relation_raw.pkl','wb'))

In [354]:
pwd

'/root/hald_autoupdate/2023_03_13'

In [355]:
sentence2verblist.get('Corneal rupture at the graft-host junction in all of our cases means the persistence of wound weakness after PK.')

[{'means': 'VBZ'}]

In [356]:
sentence2verbprototype.get('Corneal rupture at the graft-host junction in all of our cases means the persistence of wound weakness after PK.')

[{'means': 'means'}]

rules_pairs=['sentence2adv','sentence2relations','sentence2verblist','sentence2verbprototype','sentence2words']

pwd

for i in rules_pairs:
    names[i]={}
    for k in ['results/step3/rules/files/'+i+str(j)+'.pkl' for j in range(1,22)]:
        names[i].update(pickle.load(open(k,'rb')))
    for k in ['../2022_12_09/rules/files/'+i+str(j)+'.pkl' for j in range(1,22)]:
        names[i].update(pickle.load(open(k,'rb')))
    names[i].update(pickle.load(open('../2022_12_09/rules/files/'+i+'_old.pkl','rb')))
    # pickle.dump(names[i],open('files/'+i+'.pkl','wb'))

drop_all1=[]
drop_all2=[]
for i in Relation_raw_all_final_copy_all:
    sentence2verbdict={}
    for j in sentence2verblist.get(i.get('sentence')):
        sentence2verbdict[list(j.keys())[0]]=list(j.keys())[0]
    for j in sentence2verbprototype.get(i.get('sentence')):
        sentence2verbdict[list(j.values())[0]]=list(j.values())[0]
    sentence2verbdict.update(bedict)
    if i.get('openie_final'):
        openie_final=[]
        for j in i.get('openie_final'):
            openi=False
            for k in j[1].split(' '):
                if sentence2verbdict.get(k):
                    openi=True
                    break
            if openi:
                openie_final.append(j)
            else:
                drop_all1.append(j)
        # if len(openie_final) != len(i.get('openie_final')):
        #     drop_all1.append(openie_final)
        #     drop_all2.append(i.get('openie_final'))
        i['openie_final']=openie_final
#     if i.get('allennlp_final'):
#         allennlp_final=[]
#         for j in i.get('allennlp_final'):
#             allennl=False
#             for k in j[1].split(' '):
#                 if sentence2verbdict.get(k):
#                     allennl=True
#                     break
#             if allennl:
#                 allennlp_final.append(j)
#             else:
#                 drop_all2.append(j)
#         # if len(allennlp_final) != len(i.get('allennlp_final')):
#         #     drop_all1.append(allennlp_final)
#         #     drop_all2.append(i.get('allennlp_final'))
#         i['allennlp_final']=allennlp_final
        

len(drop_all1)